# Borehole Profitability Analysis

The goal of this project is to provide recommendations for an oil company that seeks to drill a new oil well. The company has done preliminary surveying of three different regions, and it has requested information about which of the three will prove the most profitable. In addition to the question of potential profit, the company has requested an analysis of the potential risk of negative profit.

The data contains information from different test sites, including unlabeled (i.e. nature of information not provided) information about  content as well as yield. The data is separated by region, and within each region results from test sites can differ greatly. Due to this, the approach chosen for this analysis is to train a linear regression model, then to make use of the bootstrap technique to establish a 95% confidence interval about drilling in a given region and to draw conclusions based on this confidence interval. 

This project could be improved in a variety of ways, notably by writing functions to make the code more laconic.

### Содержание
<a id='Содержание'></a>

[Импортирование библиотек и функций](#Импорты)

[Загрузка и просмотр данных](#Загрузка)

[Разделение данных на выборки и тренирование моделей](#Разделение_и_тренирование)

[Подготовка к расчётам](#Подготовка)

[Расчёт прибыли](#Расчёт)

[Подсчёт рисков и прибыли с помощью Bootstrap](#Бутстрепочки)

[Общие выводы](#Общий_вывод)



### Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

- Бюджет на разработку скважин в регионе — 10 млрд рублей.

- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

### Импортирование библиотек и функций
<a id='Импорты'></a>

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error




### Загрузка и просмотр данных
<a id='Загрузка'></a>

In [61]:
try:
    geo_0 = pd.read_csv('datasets/geo_data_0.csv')
    geo_1 = pd.read_csv('datasets/geo_data_1.csv')
    geo_2 = pd.read_csv('datasets/geo_data_2.csv')

except:
    geo_0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo_1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo_2 = pd.read_csv('/datasets/geo_data_2.csv')

geo_0.info()
display(geo_0.head().T)
print()
geo_1.info()
display(geo_1.head().T)
print()
geo_2.info()
display(geo_2.head().T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,0,1,2,3,4
id,txEyH,2acmU,409Wp,iJLyR,Xdl7t
f0,0.705745,1.334711,1.022732,-0.032172,1.988431
f1,-0.497823,-0.340164,0.15199,0.139033,0.155413
f2,1.22117,4.36508,1.419926,2.978566,4.751769
product,105.280062,73.03775,85.265647,168.620776,154.036647



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,0,1,2,3,4
id,kBEdx,62mP7,vyE1P,KcrkZ,AHL4O
f0,-15.001348,14.272088,6.263187,-13.081196,12.702195
f1,-8.276,-3.475083,-5.948386,-11.506057,-8.147433
f2,-0.005876,0.999183,5.00116,4.999415,5.004363
product,3.179103,26.953261,134.766305,137.945408,134.766305



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,0,1,2,3,4
id,fwXo0,WJtFt,ovLUW,q6cA6,WPMUX
f0,-1.146987,0.262778,0.194587,2.23606,-0.515993
f1,0.963328,0.269839,0.289035,-0.55376,1.716266
f2,-0.828965,-2.530187,-5.586433,0.930038,5.899011
product,27.758673,56.069697,62.87191,114.572842,149.600746


### Разделение данных на выборки и тренирование моделей
<a id='Разделение_и_тренирование'></a>

In [62]:
features_0 = geo_0.drop(columns=['id', 'product'])
target_0 = geo_0['product']

features_train_0, features_valid_0, target_train_0, target_valid_0 = (
train_test_split(features_0, target_0, test_size=0.25, random_state=12345)
)

features_1 = geo_1.drop(columns=['id', 'product'])
target_1 = geo_1['product']

features_train_1, features_valid_1, target_train_1, target_valid_1 = (
train_test_split(features_1, target_1, test_size=0.25, random_state=12345)
)

features_2 = geo_2.drop(columns=['id', 'product'])
target_2= geo_2['product']

features_train_2, features_valid_2, target_train_2, target_valid_2 = (
train_test_split(features_2, target_2, test_size=0.25, random_state=12345)
)

*Что делать с тренированием модели?*

Мне кажется, что лучшая модель получится если её трейнировать на основе всех данных из всех трёх регионов. 

Это кажется логично (больше тренировочных данных должно дать более точные предсказания), но всё-таки стоит проверить такую модель, и модели тренированные на основе региональных данных в отдельности.

Почему:
- суть данных нам не известна. Например, возможно, что отношение f0 к f1 говорит о какой-нибудь особенности региона. Тогда стоило бы строить отдельные модели.
- может из-за разных разбросов в данных регионов (или подобная такая причина), отдельные модели выдадут лучшие результаты.

Однако:
- Возможно неправильно сравнивать результаты одной модели с результатами другой. Если так поступать, хотя результаты должны быть очень похожими, они всё-таки будут различаться в какой-то мере, а тогда это уже не будет сравнение подобное с подобным.

В итоге:
- В конце концов, возьму и буду использовать модель(или модели), которая выдаст самые лучшие и точние предсказания (измеренные с помощью roc-auc, поскольку целевой признак является числительным, а не категориальным).

*Модель тренированная на данных всех регионов*

In [63]:
features_train_all = pd.concat([features_train_0, features_train_1, features_train_2])
target_train_all = pd.concat([target_train_0, target_train_1, target_train_2])
# features_train_all.info()

In [64]:
linreg_all = LinearRegression().fit(features_train_all, target_train_all)
linreg_0 = LinearRegression().fit(features_train_0, target_train_0)
pred_0 = linreg_0.predict(features_valid_0)
print(f'''
Оценки модели, тренированной на данных всех регионов:
с тренировочной выборкой (региона 0): {linreg_all.score(features_train_0, target_train_0)}
с валидационной выборкой (региона 0): {linreg_all.score(features_valid_0, target_valid_0)}

Оценки модели, тренированной на данных региона 0:
с тренировочной выборкой (региона 0): {linreg_0.score(features_train_0, target_train_0)}
с валидационной выборкой (региона 0): {linreg_0.score(features_valid_0, target_valid_0)}
''')


Оценки модели, тренированной на данных всех регионов:
с тренировочной выборкой (региона 0): 0.1954350196873944
с валидационной выборкой (региона 0): 0.19944949164638637

Оценки модели, тренированной на данных региона 0:
с тренировочной выборкой (региона 0): 0.27423906493940775
с валидационной выборкой (региона 0): 0.27994321524487786



Получается, что лучше тренировать модели для каждого региона отдельно, вместо того, чтобы использовать одну модель и тренировать её с данными из всех регионов.


*Тренирование двух остальных моделей*

In [65]:
linreg_1 = LinearRegression().fit(features_train_1, target_train_1)
pred_1 = linreg_1.predict(features_valid_1)
linreg_2 = LinearRegression().fit(features_train_2, target_train_2)
pred_2 = linreg_2.predict(features_valid_2)

print(f'''Оценки модели, тренированной на данных региона 1:
с тренировочной выборкой (региона 1): {linreg_1.score(features_train_1, target_train_1)}
с валидационной выборкой (региона 1): {linreg_1.score(features_valid_1, target_valid_1)}
''')

print(f'''Оценки модели, тренированной на данных региона 2:
с тренировочной выборкой (региона 2): {linreg_2.score(features_train_2, target_train_2)}
с валидационной выборкой (региона 2): {linreg_2.score(features_valid_2, target_valid_2)}
''')

Оценки модели, тренированной на данных региона 1:
с тренировочной выборкой (региона 1): 0.9996247760326118
с валидационной выборкой (региона 1): 0.9996233978805127

Оценки модели, тренированной на данных региона 2:
с тренировочной выборкой (региона 2): 0.19661432867329998
с валидационной выборкой (региона 2): 0.20524758386040443



In [66]:
rmse_0 = mean_squared_error(target_valid_0, pred_0)**0.5
rmse_1 = mean_squared_error(target_valid_1, pred_1)**0.5
rmse_2 = mean_squared_error(target_valid_2, pred_2)**0.5
mean_product_0_pred = pred_0.mean()
mean_product_1_pred = pred_1.mean()
mean_product_2_pred = pred_2.mean()
mean_product_0_act = target_valid_0.mean()
mean_product_1_act = target_valid_1.mean()
mean_product_2_act = target_valid_2.mean()

In [67]:
print(f'''
RMSE предсказаний об объёме и настоящего объёма для региона 0: {rmse_0}
Средний предсказанный объём сырья: {mean_product_0_pred}
Средний объём (настоящий): {mean_product_0_act}

RMSE предсказаний об объёме и настоящего объёма для региона 1: {rmse_1}
Средний предсказанный объём сырья: {mean_product_0_pred}
Средний объём (настоящий): {mean_product_1_act}

RMSE предсказаний об объёме и настоящего объёма для региона 2: {rmse_2}
Средний предсказанный объём сырья: {mean_product_0_pred}
Средний объём (настоящий): {mean_product_2_act}
''')


RMSE предсказаний об объёме и настоящего объёма для региона 0: 37.5794217150813
Средний предсказанный объём сырья: 92.59256778438035
Средний объём (настоящий): 92.07859674082927

RMSE предсказаний об объёме и настоящего объёма для региона 1: 0.893099286775617
Средний предсказанный объём сырья: 92.59256778438035
Средний объём (настоящий): 68.72313602435997

RMSE предсказаний об объёме и настоящего объёма для региона 2: 40.02970873393434
Средний предсказанный объём сырья: 92.59256778438035
Средний объём (настоящий): 94.88423280885438



#### Выводы

Исходя из результатов, можно делать некоторые выводы:

- модели не настолько хорошо предгадывают объём сырья
    - Признаки f0, f1, и f2 не определяют количество добываемого сырья, а скорее просто намекают на какие-то тенденции. Поэтому, еслм смотреть на таблицу упорядоченную по прибыли предсказанных самых прибыльных скважин, настоящий объём сырья этих скважин в целом уменьшается в таблице, но не в точном порядке и с некоторыми значительно меньше прибыльными скважинами на верху таблицы.
    - Оценки моделей не настолько высокие (с исключением региона 1).
- Диапазон объём сырья очень большой – некоторые скважины вообще ничего не дают, пока другие очень много дают.
- По объёму сырья (предсказанному и настоящему), регионы 0 и 2 более похожи друг на друга, а регион 1 отличается.
    - Модель региона 1 гораздо точнее предсказывает.
    - У скажин в регионе 1 объём сырья заметно меньше.

In [68]:
example_mean_product_0, example_mean_product_1, example_mean_product_2 = (
        [pred.mean() for pred in [pred_0, pred_1, pred_2]]
    )

print(example_mean_product_0)

92.59256778438035


### Подготовка к расчётам
<a id='Подготовка'></a>

*(всё в рублях)*

*Выявление нужных переменных*

In [70]:
pred_0 = pd.Series(pred_0, index=features_valid_0.index).sort_values(ascending=False)
pred_1 = pd.Series(pred_1, index=features_valid_1.index).sort_values(ascending=False)
pred_2 = pd.Series(pred_2, index=features_valid_2.index).sort_values(ascending=False)

In [71]:
BOREHOLE_COST = 10000000000
BARRELS_1000_YIELD = 450000
MINIMUM_1000_BARRELS = BOREHOLE_COST/BARRELS_1000_YIELD
print(f'Минампльное количество баррелей добываемых из 200 скважин (в каждом регионе): {MINIMUM_1000_BARRELS}')

Минампльное количество баррелей добываемых из 200 скважин (в каждом регионе): 22222.222222222223


*Сравнение минимального безубыточного объёма сырья с средним запасом в каждом регионе*

In [72]:
avg_barrels_profit_0 = target_0.mean() * 200 * BARRELS_1000_YIELD- BOREHOLE_COST
avg_barrels_profit_1 = target_1.mean() * 200 * BARRELS_1000_YIELD- BOREHOLE_COST
avg_barrels_profit_2 = target_2.mean() * 200 * BARRELS_1000_YIELD- BOREHOLE_COST


print(f'Доход от региона, посчитан по средному запасу скважин: \n Регион 0: {avg_barrels_profit_0} \n Регион 1: {avg_barrels_profit_1}\n Регион 2: {avg_barrels_profit_2}')

Доход от региона, посчитан по средному запасу скважин: 
 Регион 0: -1674999999.999998 
 Регион 1: -3805749999.999998
 Регион 2: -1449999999.9999971


#### Выводы

- Большой диапазон объёма сырья делает расчёт среднего дохода не показательным.
    - В расчётах ниже, ясно, что можно получить хороший результат и прибыль в любом регионе, но при расчёте среднего дохода получилась отрицательную прибыль в каждом регионе. Это наверно из-за скважин, у которых объём сырья был равен 0.
    
- Чтобы получить доход от региона, нужно, чтобы из него получились как минимум 22222.222222222223 баррелей сырья.

### Расчёт прибыли
<a id='Расчёт'></a>

*Выявление функций*

In [73]:
def profit_top_200(predicted_boreholes):
    '''Принимает или DataFrame, или Series с настоящими данными о скважинах, у которых была самая высокая предсказанная прибыль.
    Возвращает доход от всех этих скважин, учитывая при этом потраты на бурение'''
    if type(predicted_boreholes) == pd.core.frame.DataFrame:
        profit_from_pred = predicted_boreholes['product'].sum() * BARRELS_1000_YIELD - BOREHOLE_COST
    if type(predicted_boreholes) == pd.core.series.Series:
        profit_from_pred = predicted_boreholes.sum() * BARRELS_1000_YIELD - BOREHOLE_COST
    return profit_from_pred

In [74]:
top_200_pred_0 = geo_0.iloc[pred_0.head(200).index]

top_200_pred_1 = geo_1.iloc[pred_1.head(200).index]

top_200_pred_2 = geo_2.iloc[pred_2.head(200).index]

*Расчёты*

In [75]:
top_pred_profit_0_actual = profit_top_200(top_200_pred_0)
top_pred_profit_1_actual = profit_top_200(top_200_pred_1)
top_pred_profit_2_actual = profit_top_200(top_200_pred_2)

print(f'''
Настоящая прибыль от скважин, определённых моделью как самые прибыльные:
Регион 0: {top_pred_profit_0_actual}
Регион 1: {top_pred_profit_1_actual}
Регион 2: {top_pred_profit_2_actual}
''')




Настоящая прибыль от скважин, определённых моделью как самые прибыльные:
Регион 0: 3320826043.1398506
Регион 1: 2415086696.681511
Регион 2: 2710349963.5998325



#### Выводы

Поскольку скважины используемые здесь самые (предсказанные и настоящие) прибыльные, результаты подсчётов конечно большие. Однако, стоит отметить, что:

    - Настоящая прибыль, во всех регионах, меньше, чем предсказанная
    - В регионах 0 и 2, прибыль от по настоящему (не по предсказаниям) самых прибыльных значительно больше чем прибыль от скважин выбранных с помощью предсказаний

### Подсчёт рисков и прибыли с помощью Bootstrap
<a id='Бутстрепочки'></a>

In [78]:
BOOTSTRAP_SAMPLES = 1000
state = np.random.RandomState(12345)

profit_from_pred_0 = []

for i in range(BOOTSTRAP_SAMPLES):
    subsample = target_valid_0.sample(n=500, replace=True, random_state=state)
    pred = pred_0[subsample.index]
    pred = pred.sort_values(ascending=False)
    selected = target_valid_0[pred.index][:200]
    profit = profit_top_200(selected)
    profit_from_pred_0.append(profit)


profit_from_pred_1 = []

for i in range(BOOTSTRAP_SAMPLES):
    subsample = target_valid_1.sample(n=500, replace=True, random_state=state)
    pred = pred_1[subsample.index]
    pred = pred.sort_values(ascending=False)
    selected = target_valid_1[pred.index][:200]
    profit = profit_top_200(selected)
    profit_from_pred_1.append(profit)

profit_from_pred_2 = []

for i in range(BOOTSTRAP_SAMPLES):
    subsample = target_valid_2.sample(n=500, replace=True, random_state=state)
    pred = pred_2[subsample.index]
    pred = pred.sort_values(ascending=False)
    selected = target_valid_2[pred.index][:200]
    profit = profit_top_200(selected)
    profit_from_pred_2.append(profit)


In [79]:
profit_from_pred_0 = pd.Series(profit_from_pred_0)

lower_pred_0 = profit_from_pred_0.quantile(0.025)
upper_pred_0 = profit_from_pred_0.quantile(0.975)
mean_pred_0 = profit_from_pred_0.mean()

profit_from_pred_1 = pd.Series(profit_from_pred_1)

lower_pred_1 = profit_from_pred_1.quantile(0.025)
upper_pred_1 = profit_from_pred_1.quantile(0.975)
mean_pred_1 = profit_from_pred_1.mean()

profit_from_pred_2 = pd.Series(profit_from_pred_2)

lower_pred_2 = profit_from_pred_2.quantile(0.025)
upper_pred_2 = profit_from_pred_2.quantile(0.975)
mean_pred_2 = profit_from_pred_2.mean()


In [80]:
print(f'''95%-й доверительный интервал доходов выбранных бутстрепом скважин:

Регион 0: {lower_pred_0, upper_pred_0}, среднее: {mean_pred_0}
Регион 1: {lower_pred_1, upper_pred_1}, среднее: {mean_pred_1}
Регион 2: {lower_pred_2, upper_pred_2}, среднее: {mean_pred_2}
''')

print(f'''Процент убыточных выборок, посдсчитан по доходам выбранных бутстрепом скважин:

Регион 0: {(profit_from_pred_0 < 0).mean()*100}%
Регион 1: {(profit_from_pred_1 < 0).mean()*100}%
Регион 2: {(profit_from_pred_2 < 0).mean()*100}%
''')


95%-й доверительный интервал доходов выбранных бутстрепом скважин:

Регион 0: (-111215545.89049526, 909766941.5534226), среднее: 396164984.8023711
Регион 1: (78050810.7517417, 862952060.2637234), среднее: 461155817.2772397
Регион 2: (-112227625.37857565, 934562914.5511636), среднее: 392950475.17060447

Процент убыточных выборок, посдсчитан по доходам выбранных бутстрепом скважин:

Регион 0: 6.9%
Регион 1: 0.7000000000000001%
Регион 2: 6.5%



### Рекомендации:
<a id='Общий_вывод'></a>


**Самый достоверный вариант для разведки новых скважин – регион 1.**

- 95%-й доверительный интервал доходов выбранных бутстрепом скважин:
    - (78050810.75, 862952060.26)
    - среднее: 461155817.2772397
-   Процент убыточных выборок, посдсчитан по доходам выбранных бутстрепом скважин:
    - Регион 1: 0.70%


[Содержание](#Содержание)